In [49]:
from pathlib import Path
import getpass

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.plot
from rasterio import features
import datetime as dt
import time

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubStatisticalDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SentinelHubStatistical,
    Geometry,
    parse_time,
)

In [50]:
# Commented out to avoid overwriting existing config
# Comment in to create new config:

# config = SHConfig()
# config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
# config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
# config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
# config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")
config = SHConfig("cdse")

In [51]:
# We also need to define the evalscript as a Python variable
evalscript_raw = """
//VERSION=3
function setup() {
   return {
    input: ["NO2", "dataMask"],
    output: 
      {
        id: "default",
        bands: 1,
        sampleType: "FLOAT32"
      },
    mosaicking: "SIMPLE"
  };
}
function evaluatePixel(sample) {
  if (sample.dataMask == 1)  {
    return [sample.NO2];
  } else {
    return [NaN];
  }
}
"""

In [52]:
# define area of interest
cologne_coords_wgs84 = [6.8, 50.8, 7.2, 51.1]
ruhrgebiet_coords_wgs84 = [6.380946, 51.315164, 7.93203, 51.738085]
germany_coords_wgs84 = [5.866315, 47.270111, 15.041896, 55.099161]

aoi_bbox = BBox(bbox=ruhrgebiet_coords_wgs84, crs=CRS.WGS84).transform(CRS(3857))

In [53]:
# define time interval
time_range = ("2020-01-01", "2020-01-31")

start, end = pd.to_datetime(time_range[0]), pd.to_datetime(time_range[1])

daily_intervals = [
    (
        day.strftime("%Y-%m-%dT00:00:00Z"),
        day.strftime("%Y-%m-%dT23:59:59Z")
    )
    for day in pd.date_range(start, end, freq="D")
]

In [55]:
# This is defining the data we will use.
# You can list all available data collections with `DataCollection.get_available_collections()`.

data_5p = DataCollection.SENTINEL5P.define_from("5p", service_url=config.sh_base_url)

for i, daily_interval in enumerate(daily_intervals):
    if i >= 15: break
    print(f"Day {i+1}: {daily_interval[0]} to {daily_interval[1]}")

    request_raw = SentinelHubRequest(
        evalscript=evalscript_raw,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_5p,
                time_interval=daily_interval,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=aoi_bbox,
        # Resolution is defined in units of the bbox crs! Be careful with WGS84 since this will be in degrees!
        # Since we have defined our bounding box in Web mercator the resolution is in meters.
        resolution=(5000, 3500),
        config=config,
        data_folder="./data",  # We save the data in a specified folder
    )
    raw_data = request_raw.get_data(redownload=True)
    print(f"  Retrieved data shape: {raw_data[0].shape}")
    print(f"  Fraction of valid NO2 values: {np.isfinite(raw_data[0]).mean()}\n")
    print(f"  Mean of NO2 values: {raw_data[0][np.isfinite(raw_data[0])].mean()}\n")

Day 1: 2020-01-01T00:00:00Z to 2020-01-01T23:59:59Z
  Retrieved data shape: (22, 35)
  Fraction of valid NO2 values: 0.9961038961038962

  Mean of NO2 values: 0.000131761931697838

Day 2: 2020-01-02T00:00:00Z to 2020-01-02T23:59:59Z
  Retrieved data shape: (22, 35)
  Fraction of valid NO2 values: 0.0

  Mean of NO2 values: nan

Day 3: 2020-01-03T00:00:00Z to 2020-01-03T23:59:59Z


/var/folders/qg/6zlm1pjd21z1syn_q1qx1yv00000gn/T/ipykernel_25471/3149312639.py:29: RuntimeWarning: Mean of empty slice.
  print(f"  Mean of NO2 values: {raw_data[0][np.isfinite(raw_data[0])].mean()}\n")


  Retrieved data shape: (22, 35)
  Fraction of valid NO2 values: 0.0

  Mean of NO2 values: nan

Day 4: 2020-01-04T00:00:00Z to 2020-01-04T23:59:59Z
  Retrieved data shape: (22, 35)
  Fraction of valid NO2 values: 0.0

  Mean of NO2 values: nan

Day 5: 2020-01-05T00:00:00Z to 2020-01-05T23:59:59Z
  Retrieved data shape: (22, 35)
  Fraction of valid NO2 values: 0.0

  Mean of NO2 values: nan

Day 6: 2020-01-06T00:00:00Z to 2020-01-06T23:59:59Z
  Retrieved data shape: (22, 35)
  Fraction of valid NO2 values: 0.9038961038961039

  Mean of NO2 values: 0.00012765268911607563

Day 7: 2020-01-07T00:00:00Z to 2020-01-07T23:59:59Z
  Retrieved data shape: (22, 35)
  Fraction of valid NO2 values: 0.00909090909090909

  Mean of NO2 values: 0.00017923479026649147

Day 8: 2020-01-08T00:00:00Z to 2020-01-08T23:59:59Z
  Retrieved data shape: (22, 35)
  Fraction of valid NO2 values: 0.0

  Mean of NO2 values: nan

Day 9: 2020-01-09T00:00:00Z to 2020-01-09T23:59:59Z
  Retrieved data shape: (22, 35)
  Fr